##### Copyright 2021 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# Use GPUs with Apache Beam

This notebook uses the [Monte Carlo method](https://en.wikipedia.org/wiki/Monte_Carlo_method) to calculate pi (3.14159...) and demonstrate the performance difference at the same scale of sample size between different runtimes:

- Native Python code
- Jitted machine code
- On GPU (CUDA)
- Beam pipeline locally on GPU
- Beam pipeline on Dataflow with GPU

**Note**: this notebook does not work if your notebook instance does not have a GPU or the drivers are not installed. If you haven't done so, create a [new Dataflow Notebooks](https://pantheon.corp.google.com/dataflow/notebooks/list/instances) instance `With 1 NVIDIA Tesla T4` and check the option to `Install NVIDIA GPU driver automatically for me`.

More details can be found on [Dataflow support for GPUs](https://cloud.google.com/dataflow/docs/concepts/gpu-support) if you want to productionize Beam pipelines on Dataflow with GPUs.

## Dependencies

We will use [numba](https://numba.pydata.org/) to compile code in this notebook for different runtimes.

>Numba is an open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.

It also supports GPU acceleration.

**Note**: you might need to restart the kernel if this is the first time you've installed the package.

In [ ]:
%pip install numba

Let's check if the CUDA libraries are available (*if not, your notebook instance probably wasn't created with a GPU*):

In [ ]:
!find /usr/local/cuda-* -iname 'libdevice'
!find /usr/local/cuda-* -iname 'libnvvm.so'

Let's disable non-error logs for clarity.

In [ ]:
import logging
logging.getLogger().setLevel(logging.ERROR)

## Native Python & jitted machine code

Below we have defined two functions with exactly the same code:

- `python_cpu_monte_carlo_pi` is a plain native Python function that runs on the CPU.
- `machine_code_cpu_monte_carlo_pi` has an annotation `@jit(nopython=True)` that compiles the Python code into machine code that runs on the CPU too.

In [ ]:
import random
from timeit import default_timer as timer

from numba import jit


def python_cpu_monte_carlo_pi(sample_size):
    acc = 0
    for i in range(sample_size):
        x = random.random()
        y = random.random()
        if (x * x + y * y) <= 1.0:
            acc += 1
    return 4.0 * acc / sample_size


@jit(nopython=True)
def machine_code_cpu_monte_carlo_pi(sample_size):
    acc = 0
    for i in range(sample_size):
        x = random.random()
        y = random.random()
        if (x * x + y * y) <= 1.0:
            acc += 1
    return 4.0 * acc / sample_size

Let's choose a sample size (100,000,000) as a constant computation complexity between both runs.

The most expensive yet parallelizable part of the computation is generating the random numbers. Neither function optimizes that part though.

In [ ]:
sample_size = 100_000_000

### Performance of native Python code

It should take ~40 seconds for native Python code to calculate pi with a sample size of 100,000,000.

**Note**: The performance might vary from run to run. It might also vary between different notebook instances if they are configured differently. The same applies to other runtimes.

In [ ]:
start = timer()
pi = python_cpu_monte_carlo_pi(sample_size)
dt = timer() - start
print(f'Monte Carlo pi calculated as {pi} in {dt} s.')

### Performance of jitted machine code

It should only take a little over 1 second for jitted (the first time execution is a bit slower: ~2 seconds, because the source code is not jitted yet) machine code to calculate pi with a sample size of 100,000,000.

After jitting the Python code, let's use `%timeit` to run the code multiple times to measure the performance.

In [ ]:
# First run is slower because the code is not jitted.
start = timer()
pi = machine_code_cpu_monte_carlo_pi(sample_size)
dt = timer() - start
print(f'Monte Carlo pi calculated as {pi} in {dt} s.')

# Run the jitted code multiple times to measure performance.
%timeit machine_code_cpu_monte_carlo_pi(sample_size)

## On GPU (CUDA)

In the below example, we have rearranged the native Python function into two parts:

- The first part, `gpu_monte_carlo_pi_sampler`, which generates random points and aggregate counts for a subset of the target sample size, is executed on the GPU.
- The second part, `calculate_pi`, which aggregates value from all sub sample sets and calculates pi, is compiled into machine code and executed on the GPU.

**Note**:`njit` is similar to `jit` but for `numpy`.

The example code uses only 1 grid, 80 blocks and 64 threads per block.

In [ ]:
from numba import cuda, njit
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
import numpy as np


@cuda.jit
def gpu_monte_carlo_pi_sampler(rng_states, sub_sample_size, acc):
    """Uses GPU to sample random values and accumulates the sub count
    of values within the circle.
    """
    pos = cuda.grid(1)
    if pos < acc.shape[0]:
        sub_acc = 0
        for i in range(sub_sample_size):
            x = xoroshiro128p_uniform_float32(rng_states, pos)
            y = xoroshiro128p_uniform_float32(rng_states, pos)
            if (x * x + y * y) <= 1.0:
                sub_acc += 1
        acc[pos] = sub_acc


@njit(fastmath=True)
def calculate_pi(acc, sample_size):
    """Uses machine code on CPU to aggregate and calculate pi since there
    is less parallelism here.
    """
    return 4 * np.sum(acc) / sample_size

threadsperblock = 64
blocks = 80
# An 1-d array on host to hold accumulated sub count of points in the circle.
h_acc = np.zeros(threadsperblock * blocks, dtype=np.float32)
# Copy the numpy array from host (CPU) to device (GPU) to avoid back-and-forth copies.
d_acc = cuda.to_device(h_acc)
rng_states = create_xoroshiro128p_states(threadsperblock * blocks, seed=1)
# Each thread should only generate sub_sample_size of random points.
sub_sample_size = sample_size // d_acc.shape[0]

### Performance of on GPU (CUDA)

It should take ~5ms (the first time execution is a bit slower: ~0.5 seconds, because the source code is not jitted yet) to calculate pi on a GPU using above configuration with a sample size of 100,000,000.

It's **8000 times faster** than native Python on CPU and **200 times** faster than machine code on CPU. And we haven't even pushed the single GPU to its limit.

In [ ]:
# First run is slower because the code is not jitted.
start=timer()
gpu_monte_carlo_pi_sampler[blocks, threadsperblock](rng_states, sub_sample_size, d_acc)
# Copy the array on device back to host.
d_acc.copy_to_host(h_acc)
pi = calculate_pi(h_acc, sample_size)
dt = timer() - start
print(f'Monte Carlo pi calculated as {pi} in {dt} s.')

# Run the jitted code multiple times to measure performance.
%timeit gpu_monte_carlo_pi_sampler[blocks, threadsperblock](rng_states, sub_sample_size, d_acc)

## Running a Beam pipeline locally on GPU

It might not be obvious why you need to wrap your code that can already run on a GPU
into an Apache Beam pipeline.

The advantage of Beam with GPU is that you can later run the pipeline on Dataflow or any other runner/clusters
so that you can distribute/scale the work to as many GPUs as you want.

Note: you might need to configure worker machines so that they have GPU devices and drivers available.
For example, [using GPUs with Dataflow](https://cloud.google.com/dataflow/docs/guides/using-gpus).
You also need to constrain your GPU usages to work within the hardware limit. See
[GPUs and worker parallelism](https://cloud.google.com/dataflow/docs/concepts/gpu-support#gpus_and_worker_parallelism).

In the below example, we build a Beam pipeline with code similar to the plain [on-GPU example](#On-GPU-(CUDA)) and run the pipeline locally on this notebook instance.

First, we create a pipeline instance with options that utilize all CPU cores to schedule work when running the pipeline.

In [ ]:
from typing import Tuple

import apache_beam as beam
from apache_beam.options import pipeline_options
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.runners.interactive import interactive_beam as ib


options = pipeline_options.PipelineOptions(
    direct_num_workers=0,  # default threads/subprocess to the number of cores on this machine
    direct_running_mode='multi_threading')
p = beam.Pipeline(InteractiveRunner(), options=options)

Then, we define a `DoFn` as a `Sampler` that uses the GPU to process elements.

Each element is a tuple of 2 `int` values:

- first value: the seed of a random number generator
- second value: the size of sample values to be generated

The `DoFn` itself runs as native Python code on the CPU while the inner logic of `gpu_monte_carlo_pi_sampler` runs on GPU.

You can also see that to avoid back-and-forth copying of the result array, `cuda.to_device()` copies the array to GPU, then `copy_to_host` returns the array on GPU back to the host.

In [ ]:
class Sampler(beam.DoFn):
    def __init__(self, blocks=80, threads_per_block=64):
        # Uses only 1 cuda grid with below config.
        self.blocks = blocks
        self.threads_per_block = threads_per_block
    
    def setup(self):
        import numpy as np
        # An array on host as the prototype of arrays on GPU to
        # hold accumulated sub count of points in the circle.
        self.h_acc = np.zeros(
            self.threads_per_block * self.blocks, dtype=np.float32)

    def process(self, element: Tuple[int, int]):
        from numba import cuda
        from numba.cuda.random import create_xoroshiro128p_states
        from numba.cuda.random import xoroshiro128p_uniform_float32
        
        @cuda.jit
        def gpu_monte_carlo_pi_sampler(rng_states, sub_sample_size, acc):
            """Uses GPU to sample random values and accumulates the sub count
            of values within a circle of radius 1.
            """
            pos = cuda.grid(1)
            if pos < acc.shape[0]:
                sub_acc = 0
                for i in range(sub_sample_size):
                    x = xoroshiro128p_uniform_float32(rng_states, pos)
                    y = xoroshiro128p_uniform_float32(rng_states, pos)
                    if (x * x + y * y) <= 1.0:
                        sub_acc += 1
                acc[pos] = sub_acc

        rng_seed, sample_size = element
        d_acc = cuda.to_device(self.h_acc)
        sample_size_per_thread = sample_size // self.h_acc.shape[0]
        rng_states = create_xoroshiro128p_states(self.h_acc.shape[0], seed=rng_seed)
        gpu_monte_carlo_pi_sampler[self.blocks, self.threads_per_block](
            rng_states, sample_size_per_thread, d_acc)
        yield d_acc.copy_to_host()

We can divide the work to 100 samplers. In a distributed environment, each sampler can run on a different machine with its own GPU(s).

Here for simplicity, we run the pipeline locally and all samplers would share the same GPU on this notebook instance.

We start the pipeline by creating a PCollection of 100 tuples of random number seeds (from 0 to 99) and sample size per sampler (1,000,000).

Then we let the `Sampler DoFn` take these tuples as inputs to generate sample values.

Each sampler has `threads_per_block` * `blocks` = **5120 threads** running in parallel
**on the GPU**. 

And we have **100 samplers** running concurrently scheduled by Beam **on all CPU cores** on the machine running this notebook.

In the collected data, you can see the aggregated counts of "random points within the circle" for each GPU thread by each sampler.

In [ ]:
sampler_count = 100
sample_size_per_sampler = sample_size // sampler_count

samplers_per_gpu_thread = (p 
                           | beam.Create([(i, sample_size_per_sampler) for i in range(sampler_count)])
                           | beam.ParDo(Sampler()))

ib.collect(samplers_per_gpu_thread)

Everything below runs as native Python code on the CPU.

To calculate pi, we need to aggregate all values from all GPU threads for each sampler.

**Note**: we use numpy to sum the values (`np.float32`) and then convert them back to the native Python type (`int`).

In [ ]:
def np_sum(x):
    import numpy as np
    return np.sum(x).astype(int).item()

acc_per_sampler = samplers_per_gpu_thread | beam.Map(np_sum).with_output_types(int)

# Sum up per-gpu-thread count of values falling into the circle of Monte Carlo pi calculation for each sampler.
ib.show(acc_per_sampler)

Then we combine all values from all samplers.

In [ ]:
sum_acc = acc_per_sampler | beam.CombineGlobally(sum)

# Sum up the count of values falling into the circle of Monte Carlo pi calculation from all samplers.
ib.show(sum_acc)

Once we have all values aggregated, we can use the formula to calculate and print pi.

The visualization shows the pipeline graph. Let's make sure it's correctly constructed and doesn't have corrupted states from notebook executions.

**Note**: if the graph is mixed with transforms that are applied by out-of-order execution and re-execution of notebooks, please re-execute the code
from where the pipeline is created or restart the kernel and re-execute the whole notebook.

In [ ]:
class CalculatePi(beam.DoFn):
    def __init__(self, sample_size):
        self.sample_size = sample_size
    
    def process(self, count: int):
        yield 4 * count / self.sample_size

calculated_pi = sum_acc | beam.ParDo(CalculatePi(sample_size))
            
ib.show_graph(p)

### Performance of Beam pipeline locally on GPU

It should take a few seconds to calculate pi using Beam on GPU with a sample size of 100,000,000.

Though it's not as fast as GPU + machine code executed on a single machine, it provides the scalability to run the code
on distributed systems, and the performance is almost on par with pure machine code on a single machine. You can also improve the
performance further by compiling those transforms written in native Python code into machine code with jit/njit.

The example has demonstrated how to write a Beam pipeline using a GPU, the performance increment when using a GPU compared to native Python code on CPU, and the small overhead of a Beam pipeline on a local machine.

In [ ]:
ib.recordings.clear(p)
start=timer()
print(f'Monte Carlo pi calculated as: {ib.collect(calculated_pi)[0][0]} in {timer() - start} s.')

## Running a Beam pipeline on Dataflow with GPU

Normally, you can [manage Python pipeline dependencies](https://beam.apache.org/documentation/sdks/python-pipeline-dependencies) with some extra pipeline options such as `requirements_file`, `extra_package` and `setup_file`. However, you have to create a [custom container](https://cloud.google.com/dataflow/docs/guides/using-custom-containers) specifically [configured](https://cloud.google.com/dataflow/docs/guides/using-gpus#building_a_custom_container_image) to instruct Dataflow to execute your pipeline on GPUs.

From this notebook, you can create a custom container that supports [CUDA](https://developer.nvidia.com/cuda-toolkit) and use it to run the pipeline on Dataflow with GPU.

- The Monte Carlo pi example requires additional dependencies: `numba` and `numpy` from PYPI.
- You can use `nvidia-tesla-t4` GPUs when running the pipeline on Dataflow.
- You can select a Beam SDK version. The example uses 2.33.0.
- The example uses Python 3.7. This should match the version used in the custom container. Check the Python version in the notebook:

In [ ]:
!python --version

Make sure the service account running this notebook instance has minimum [permissions and roles](https://cloud.google.com/container-registry/docs/access-control#permissions_and_roles) to build and publish container images to the current project's container registry.

In [ ]:
# Check gcloud configuration
!gcloud iam service-accounts describe $(gcloud config get-value account)
!gcloud projects get-iam-policy $(gcloud config get-value project)  \
  --flatten="bindings[].members" \
  --format='table(bindings.role)' \
  --filter="bindings.members:$(gcloud config get-value account)"

# Enable the container registry service
!gcloud services enable containerregistry.googleapis.com

# Configure docker
!yes | gcloud auth configure-docker

Create a temporary directory and a dockerfile as the context to build a container image.

In [ ]:
!mkdir -p /home/jupyter/.cc_gpu

The dockerfile follows this [example](https://cloud.google.com/dataflow/docs/guides/using-gpus#installing_a_specific_python_version) with a little change based on the SDK version and Python version. It uses the `devel` versioned image so that `nvmm` is available to compile the `numba.cuda.jit` code. Make sure the CUDA version from the [base image from NVIDIA](https://catalog.ngc.nvidia.com/orgs/nvidia/containers/cuda/tags) matches the installed version on VMs used by Dataflow. If something went wrong, refer [Debug with a standalone VM](https://cloud.google.com/dataflow/docs/guides/using-gpus#debug_with_a_standalone_vm) to check the CUDA version using the command `nvidia-smi`.

In [ ]:
%%writefile /home/jupyter/.cc_gpu/Dockerfile
FROM nvcr.io/nvidia/cuda:11.0.3-devel-ubuntu20.04

# Avoid tzdata hanging the docker build.
ARG DEBIAN_FRONTEND=noninteractive

RUN \
    apt update \
    && apt install -y software-properties-common \
    && add-apt-repository ppa:deadsnakes/ppa \
    && apt remove -y software-properties-common \
    && apt autoremove -y \
    && apt update \
    && apt install -y curl python3.7 python3-distutils \
    && update-alternatives --install /usr/bin/python python /usr/bin/python3.7 10 \
    && curl https://bootstrap.pypa.io/get-pip.py | python \
    && python -m pip install --upgrade pip \
    && pip install --no-cache-dir "apache-beam[gcp]==2.33.0" numba numpy pandas\
    && pip check

# Copy the Apache Beam worker dependencies from the Beam Python 3.7 SDK image.
COPY --from=apache/beam_python3.7_sdk:2.33.0 /opt/apache/beam /opt/apache/beam

# Set the entrypoint to Apache Beam SDK worker launcher.
ENTRYPOINT [ "/opt/apache/beam/boot" ]

Use [Cloud Build](https://cloud.google.com/build) to build the container image and publish it to a container registry that your Dataflow service has access to. The example publishes it to the currently configured project's container registry. Change the command below accordingly if you have another publish destination in mind.

**Note**: Building and publishing a fresh container image takes a relatively long time. You don't have to re-execute it if there is no change to the dockerfile.

**Warning**: This notebook instance comes with docker (from its host VM) to support xLang transforms such as [Beam SQL](https://beam.apache.org/documentation/dsls/sql/overview/), **do not** use it to build docker containers.

In [ ]:
%%bash -e
project=$(gcloud config get-value project)
cd /home/jupyter/.cc_gpu
gcloud builds submit --tag gcr.io/$project/cc_gpu:latest --timeout=20m

The container image `gcr.io/$project/cc_gpu:latest` should be available to use.

In [ ]:
!gcloud container images list | grep cc_gpu

Configure the pipeline options and add additional transforms.

In [ ]:
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import DebugOptions
from apache_beam.runners import DataflowRunner

import google.auth


_, PROJECT = google.auth.default()
GCS_BUCKET = 'gs://your-GCS-bucket'

# Write the result to a file on Cloud Storage
_ = calculated_pi | beam.io.WriteToText(f'{GCS_BUCKET}/staging/calculated_pi')

options = pipeline_options.PipelineOptions(
  project=PROJECT,
  region='us-central1',
  staging_location=f'{GCS_BUCKET}/staging',
  temp_location=f'{GCS_BUCKET}/temp',
  sdk_container_image=f'gcr.io/{PROJECT}/cc_gpu:latest',
  machine_type='n1-standard-4',
  disk_size_gb=50)
options.view_as(DebugOptions).add_experiment('worker_accelerator=type:nvidia-tesla-t4;count:1;install-nvidia-driver')
options.view_as(DebugOptions).add_experiment('use_runner_v2')

Now run the pipeline on Dataflow.

In [ ]:
pipeline_result = DataflowRunner().run_pipeline(p, options=options)

from IPython.core.display import display, HTML
url = ('https://console.cloud.google.com/dataflow/jobs/%s/%s?project=%s' % 
      (pipeline_result._job.location, pipeline_result._job.id, pipeline_result._job.projectId))
display(HTML('Click <a href="%s" target="_new">here</a> for the details of your Dataflow job!' % url))

pipeline_result.wait_until_finish()

Check the result once the job is done.

In [ ]:
!gsutil cat {GCS_BUCKET}/staging/calculated_pi*